# HMI for vSFL-Harvester 
This is a KWH HMI which allows to request the vSFL-havester's data, all necessary modules are imported into the script, including the S³I library.harvester's data. This Notebook acts as an **HMI** in terms of the S³I and a **client** in terms of OAuth authentication. Enter the id of your HMI with the corresponding secret as *hmi* in this script to make this notebook to your HMI. Running this notebook, you will authorize your HMI to call up the harvester's data. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
import s3i
import json
import uuid
import jwt
import time
import os
import base64 
import getpass
import requests
import collections
from threading import Thread, Event
import sys
from tools import print_with_timestamp, check_message_encryption, yes, no## Configure the notebook

## Configure the notebook
In order to use the S³I this notebook needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [ ]:
print_with_timestamp("Assign a client to this notebook. (The id of your HMI)")
hmi_id = input('[S3I]: Please enter your HMI id:').strip('," ')
hmi_secret = getpass.getpass('[S3I]: Please enter your HMI secret:').strip('," ')
print_with_timestamp("HMI id and secret are set")

Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the harvester's data on your behalf.

In [ ]:
print_with_timestamp("DEMO vSFL-harvester, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=hmi_id,
                                             client_secret=hmi_secret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token'''
parsed_username = jwt.decode(access_token, verify=False)["preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

After the token has been received, it can be used to query the endpoint of the vSFL- harvester. To query the vSFL-harvester in the S³I Directory, you must have the rights to view the vSFL-harvester. With the creation of your S³I account you should have got this right. In case of problems please contact the S³I team (s3i@kwh40.de).

Besides the endpoint of the harvester, the endpoint of this HMI is also needed to add as "ReplyToEndpoint" field in the request. The endpoint of this HMI is also queried from the directory, as it may have changed. By querying its own endpoint from the directory, the HMI is independent of changes made to its own endpoint. These changes then only need to be listed in the directory.

In [ ]:
"""authentication with JWT in S3I Directory """

s3i_directory = s3i.Directory(
    s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)

print_with_timestamp("Authentication with Token in S3I Directory successful")

""" query the hmi's endpoint"""

hmi_endpoints = s3i_directory.queryThingIDBased(hmi_id+"/attributes/allEndpoints")
""" find the hmi's endpoint in broker"""
for endpoint in hmi_endpoints:
    if "s3ib" in endpoint:
        hmi_endpoint = endpoint

""" find the thingId of the vSFL-harvester"""
harvester_id = s3i_directory.queryAttributeBased(
    "name", "vSFL-Harvester")[0]["thingId"]

""" query the vSFL-Harvester's endpoint in broker"""
vSFL_harvester_endpoints = s3i_directory.queryThingIDBased(harvester_id+"/attributes/allEndpoints")
harvester_endpoint = ""
for endpoint in vSFL_harvester_endpoints:
    if "s3ib" in endpoint:
        harvester_endpoint = endpoint 

## MessageId Storage
The messageIds storage stores all message ids from the messages which have been sent. If a message arrives it is checked against these ids to see, if it is a response to one of these messages. If it is not a response to one a request send from this notebook, the message won't be parsed.

In [ ]:
messageIds = list()

# >>> *Run All below* from here to access on the vSFL-Harvester
______________________________________________________________________________________

## Functionality 1: Send a felling job (cutting length for stem sections) to vSFL-harvester
To send a felling job to the vSFL-Harvester, we use a S3I-B-ServiceRequest to transmit the job.
### 1.a) Prepared the request. A S3I-B-*ServiceRequest* is instantiated and filled in.

In [ ]:
print_with_timestamp("Prepare the ServiceRequest.")
ser_req = s3i.ServiceRequest()

felling_job = {
        "fellingJob": {
            "class": "fml40::FellingJob",
            "name": "FellingJob4711",
            "subFeatures": [
                {
                    "class": "ml40::Location",
                    "longitude": "7.99678440031724",
                    "latitude": "51.4529725978349"
                },
                {
                    "class": "ml40::Shared",
                    "name": "Zielbestand",
                    "targets": [
                        "s3i:cb603702-83a0-49c8-a398-80ab37b56c2a"
                    ]
                },
                {
                    "class": "fml40::Assortment",
                    "grade": "is",
                    "name": "Industrieholz kurz",
                    "subFeatures": [
                        {
                            "class": "fml40::TreeType",
                            "name": "Spruce",
                            "conifer": "true"
                        },
                        {
                            "class": "fml40::ThicknessClass",
                            "name": "1a-1b"
                        },
                        {
                            "class": "fml40::HarvestingParameters",
                            "cuttingLengths": "1.5"
                        }
                    ]
                },
                {
                    "class": "fml40::Assortment",
                    "grade": "fl",
                    "name": "Stammholz Abschnitte",
                    "subFeatures": [
                        {
                            "class": "fml40::TreeType",
                            "name": "Spruce",
                            "conifer": "true"
                        },
                        {
                            "class": "fml40::ThicknessClass",
                            "name": ">=2a"
                        },
                        {
                            "class": "fml40::WoodQuality",
                            "name": "B-C"
                        },
                        {
                            "class": "fml40::HarvestingParameters",
                            "name": "Ernte Parameter",
                            "cuttingLengths": "2"
                        }
                    ]
                }
            ]
        }
    }


ser_req.fillServiceRequest(
            senderUUID = hmi_id,
            receiverUUID = [harvester_id],
            sender_endpoint = hmi_endpoint,
            serviceType = "AcceptsFellingJobs/acceptFellingJob",
            parameters = felling_job,
            msgUUID = "s3i:{}".format(uuid.uuid4())
)
print_with_timestamp("ServiceRequest prepared:")
print(json.dumps(ser_req.msg, indent=2))

### 1.b) Send the S3I-B-serviceRequest via S3I-Broker API
The HMI requests a new token to establish a connection to the broker and sends the request to the harvester's endpoint. It then checks incoming responses.

In [ ]:
print_with_timestamp("Sending the serviceRequest to the vSFL-harvester")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}

"""Store the message id into message storage"""
messageIds.append(ser_req.msg["identifier"])

"""Send the serviceRequest via S3I-Broker API"""
service_req_response = requests.post(url="https://broker.s3i.vswf.dev/{}".format(harvester_endpoint),
                                     data=json.dumps(ser_req.msg), headers=headers)
print_with_timestamp(service_req_response.text)

### 1.c) Receive the harvester's S3I-B-ServiceReply via S3I-Broker API

In [ ]:
def receive(msg_type):
    access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
    headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
    response = requests.get(url="https://broker.s3i.vswf.dev/{}".format(hmi_endpoint) , headers=headers)
    msg_json = ""
    value_json = ""
    json_acceptable_string = response.text.replace("'", "\"")
    if json_acceptable_string:
        if check_message_encryption(response.text.strip('"')) == "pgp":
            print_with_timestamp("You received a PGP message but this notebook can not decrypt PGP messages. Use the 03_inbox notebook to receive PGP messages.")
            print_with_timestamp("PGP Message: " + response.text)
            repeat(msg_type)
        else: 
            msg_json = json.loads(json_acceptable_string)
            if msg_json["replyingToMessage"] in messageIds and msg_json["messageType"] == msg_type:
                if msg_type == "serviceReply":
                    value_json = msg_json["results"]
                elif msg_type == "getValueReply":
                    value_json = msg_json["value"]
                elif msg_type == "setValueReply":
                    value_json = msg_json["ok"]
                messageIds.remove(msg_json["replyingToMessage"])
            else:
                repeat(msg_type)
                
    else:
        print_with_timestamp("The harvester did not respond yet.")
        repeat(msg_type)
    return msg_json, value_json

def repeat(msg_type):
    decision = input("[S3I] Do you want to check for new messages again? [j/n]")
    if decision in yes:
        receive(msg_type)
    elif decision in no:
        print_with_timestamp("You do not want to check for more messages. If you want to check for new messages, just execute this cell again (Run button or SHIFT+RETURN)")
    else:
        print_with_timestamp("I could not understand your response. If you want to check for new messages, just execute this cell again (Run button or SHIFT+RETURN)")

In [ ]:
print_with_timestamp("Checking the harvester's response")
ser_reply, ser_reply_value = receive(msg_type="serviceReply")

In [ ]:
if ser_reply:
    ser_reply = json.loads(json.dumps(ser_reply), object_pairs_hook=collections.OrderedDict)
    print_with_timestamp("You received a message: " + json.dumps(ser_reply, indent=2))
    

In [ ]:
if ser_reply_value:
    ser_reply_value = json.loads(json.dumps(ser_reply_value), object_pairs_hook=collections.OrderedDict)
    print_with_timestamp("Your requested value is: " + json.dumps(ser_reply_value, indent=2))

## Functionality 2: Send the adjusted cutting length to the vSFL-harvester 
To change a cutting length in the vSFL-Harvester, we use a S3I-B-SetValueRequest to transmit the command.

### 2.a) Prepare a S3I-B-SetValueRequest manually

In [ ]:
print_with_timestamp("Prepare the SetValueRequest.")
attribute_path = input("[S3I]: Please enter the attribute path, whose value is going to be changed: \n e.g. [s3i:74c37a5c-575e-46c9-a854-25067cf2b69a/nfeatures/nJobList/nsubFeatures/nFellingJob4711/nsubFeatures/nStammholz Abschnitte/nsubFeatures/nErnte Parameter/ncuttingLengths] \n")
new_value = input("[S3I]: Please enter the new value (e.g. 3): ")
set_value_req = {
    "sender": hmi_id,
    "identifier": "s3i:{}".format(uuid.uuid4()),
    "receivers": [harvester_id],
    "messageType": "setValueRequest",
    "replyToEndpoint": hmi_endpoint,
    "attributePath": attribute_path,
    "newValue": new_value
}
print_with_timestamp("SetValueRequest prepared:")
print(json.dumps(set_value_req, indent=2))

### 2.b) Send the S3I-B-SetValueRequest via S3I-Broker API
The HMI requests a new token to establish a connection to the broker and sends the request to the harvester's endpoint. It then checks incoming responses.

In [ ]:
print_with_timestamp("Sending the SetValueRequest to the vSFL-harvester")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}

"""Store the message id into message storage"""
messageIds.append(set_value_req["identifier"])

"""Send the SetValueRequest via S3I-Broker API"""
set_value_req_response = requests.post(url="https://broker.s3i.vswf.dev/{}".format(harvester_endpoint),
                                      data=json.dumps(set_value_req), headers=headers)
print_with_timestamp(set_value_req_response.text)

### 2.c) Receive the harvester's S3I-B-SetValueReply via S3I-Broker API

In [ ]:
print_with_timestamp("Checking the harvester's response")
set_value_reply, set_value_reply_value = receive(msg_type="setValueReply")

In [ ]:
if set_value_reply:
    set_value_reply = json.loads(json.dumps(set_value_reply), object_pairs_hook=collections.OrderedDict)
    print_with_timestamp("You received a message: " + json.dumps(set_value_reply, indent=2))

In [ ]:
if set_value_reply_value:
    set_value_reply_value = json.loads(json.dumps(set_value_reply_value), object_pairs_hook=collections.OrderedDict)
    print_with_timestamp("Your requested value is: " + json.dumps(set_value_reply_value, indent=2))

## Functionality 3: Display the production data from the vSFL-harvester 
To display the production data which is generated by the vSFL-harvester, we use a S3I-B-GetValueRequest.

### 3.a) Prepare a S3I-B-GetValueRequest 

In [ ]:
print_with_timestamp("Prepare the GetValueRequest.")
get_value_req = s3i.GetValueRequest()
attribute_path = input("[S3I]: Please enter the attribute path, whose value is going to be queried: \ne.g. [s3i:74c37a5c-575e-46c9-a854-25067cf2b69a/nfeatures/nJobList/nsubFeatures/nFellingJob4711/nsubFeatures/nStammholz Abschnitte/nsubFeatures/nErnte Parameter/ncuttingLengths] \n [s3i:74c37a5c-575e-46c9-a854-25067cf2b69a/nfeatures/nJobList/nsubFeatures/nFellingJob4711] \n")
get_value_req.fillGetValueRequest(senderUUID=hmi_id, receiverUUID=[harvester_id], sender_endpoint=hmi_endpoint,
                                  attributePath=attribute_path, msgUUID="s3i:{}".format(uuid.uuid4()))
print_with_timestamp("GetValueRequest prepared:")
print(json.dumps(get_value_req.msg, indent=2))

### 3.b) Send the S3I-B-getValueRequest via S3I-Broker API
The HMI requests a new token to establish a connection to the broker and sends the request to the harvester's endpoint. It then checks incoming responses.

In [ ]:
print_with_timestamp("Sending the getValueRequest to the vSFL-harvester")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
stop_event = Event()

def send_get_value_req_in_loop():
    """Store the message id into message storage"""
    messageIds.append(get_value_req.msg["identifier"])
    """Send the serviceRequest via S3I-Broker API"""
    get_value_req_response = requests.post(url="https://broker.s3i.vswf.dev/{}".format(harvester_endpoint),
                                         data=json.dumps(get_value_req.msg), headers=headers)
    print_with_timestamp(get_value_req_response.text)

### 3.c) Receive the harvester's S3I-B-ServiceReply via S3I-Broker API

In [ ]:
def receive_get_value_reply_in_loop():
    get_value_reply, get_value_reply_value = receive(msg_type="getValueReply")
    get_value_reply_value = json.loads(json.dumps(get_value_reply_value))
    print("\r[S3I]: Reply: {}".format(get_value_reply_value), end="")
    sys.stdout.flush()

### 3.d) Display the getValueReply on this dashboard

In [ ]:
def display_in_loop():
    while True:
        if stop_event.is_set():
            break 
        send_get_value_req_in_loop()
        receive_get_value_reply_in_loop()
        time.sleep(1)
            
thread = Thread(target=display_in_loop)
thread.start()
while True: 
    stop_thread = input("[S3I]: Do you want to stop the dashboard? (j/n): ")
    if stop_thread in yes:
        stop_event.set()
        break 
    else:
        time.sleep(1)    